In [149]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from implementations import *

# from test_utils import test

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Run de A a Z - Ridge Regression 

### Loading  data 

In [150]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("dataset")


### Extract labels 

In [151]:
labels = np.genfromtxt("dataset/x_test.csv", delimiter=",", dtype = str, max_rows=1)
labels = np.delete(labels,0) # delete the label 'id' as we dont have it in x_train and x_test

### Cleaning and feature selection

Due to the high number of features with all different exceptions values and linearity, we decided to select manually the features we found relevant. The list below is the name of the feature, the exception values that we have to remplace, and wheter we remplace by majority (True) or by the mean (False)

In [152]:
label_list = [("MSCODE", [], True),
            ("_HCVU651", [9], True),
            ("_RFHYPE5", [9], True),
            ("_RFCHOL", [9], True),
            ("_RACE",[9], True),
            ("_BMI5",[], False),
            ("_EDUCAG",[9], True),
            ("_INCOMG",[9], True),
            ("_SMOKER3",[9], True),
            ("_DRNKWEK",[99900], True),
            ("_FRUTSUM",[], True),
            ("_VEGESUM",[], True),
            ("PA1MIN_",[], True),
            ("GENHLTH",[7,9], False),
            ("CHECKUP1",[7,9], False),
            ("MENTHLTH",[88, 77, 99], False),
            ("BPHIGH4",[7,9], True),
            ("BPMEDS",[7,9], True),
            ("TOLDHI2",[7,9], True),
            ("CHCOCNCR",[7,8,9], True),
            ("DIABETE3",[7,8,9], True),
            ("SEX",[], True),
            ("QLACTLM2",[7,9], True),
            ("AVEDRNK2",[77, 99], False),
            ("EXERANY2",[7,9], True),
            ("SHINGLE2", [7,9], True),
            ("LMTJOIN3", [7,9], True),
            ("CVDASPRN", [7,9], True)
            ]
cleaned_x_train = clean_data(labels, label_list, x_train)

## Cleaning in another way

In [153]:
threshold = 0.7  # You want to drop columns with > 50% NaNs
column_count = x_train.shape[0]
max_nan_count = threshold * column_count

# Create a mask to identify columns with too many NaNs
nan_mask = np.sum(np.isnan(x_train), axis=0) <= max_nan_count

# Use the mask to select the columns with fewer NaNs
x_train_filtered = x_train[:, nan_mask]
x_test_filtered = x_test[:, nan_mask]

# drop the 

In [ ]:
nan_mask

In [125]:
len(x_train_filtered[0])

202

In [79]:
# want to see which labels are way to small

for label in label_list:
    l, _, _ = label
    id = np.where(labels == l)[0][0]
    x = x_train[:][id]

    # Count the number of NaN values
    nans= np.count_nonzero(np.isnan(x))
    nan_perc = nans/len(x) *100
    print(nan_perc)

46.728971962616825
46.728971962616825
42.05607476635514
32.398753894081
38.940809968847354
41.43302180685358
44.54828660436137
41.74454828660436
41.74454828660436
44.85981308411215
50.155763239875384
42.99065420560748
46.10591900311526
42.679127725856695
36.7601246105919
38.006230529595015
43.925233644859816
42.05607476635514
50.77881619937694
38.31775700934579
39.56386292834891
45.17133956386293
30.8411214953271
42.679127725856695
42.36760124610592
47.35202492211838
52.024922118380054
47.66355140186916


### Training

#### Ridge regression

In [163]:
lambda_ = 0.0001
degree = 25
## tx is cleaned data 
poly = build_poly(x_train_filtered,degree)
w , loss = ridge_regression(y_train,poly,lambda_)

KeyboardInterrupt: 

#### Mean squared errors gd

In [176]:
gamma = 0.00001
max_iters = 2
w_initial = np.zeros(cleaned_x_train.shape[1])

w, loss = mean_squared_error_gd(y_train, cleaned_x_train, w_initial, max_iters, gamma)

#### Mean squared error sgd

In [160]:
gamma = 0.00001
max_iters = 100
w_initial = np.zeros(cleaned_x_train.shape[1])

w, loss = mean_squared_error_sgd(y_train, cleaned_x_train, w_initial, max_iters, gamma)

#### Least squared error

In [154]:
w, loss = least_squares(y_train, cleaned_x_train)

In [122]:
print(len(label_list))

28


In [170]:
print(w)
print(loss)
print(len(w))

[4.29695599e+01 2.51145669e+01 3.26484113e+01 3.16032914e+01
 4.05931422e+01 6.39574453e+02 7.05774129e+01 9.55269539e+01
 7.35393282e+01 3.62905491e+04 3.00867864e+01 4.64124920e+01
 2.09206344e+04 5.64303677e+01 3.70916489e+01 2.53904536e+02
 5.19182016e+01 2.52404526e+01 3.86315896e+01 4.45972177e+01
 6.62916371e+01 3.37707816e+01 4.18467877e+01 8.99539567e+01
 2.73960169e+01 4.63646235e+01 4.37618949e+01 4.65296156e+01]
635309514315469.8
28


#### Testing

In [171]:
#clean up x test

cleaned_x_test = clean_data(labels, label_list, x_test)


In [147]:
poly_test = build_poly(cleaned_x_test,degree)
poly_test.shape
                                 

(109379, 29)

#### Prediction

# notes for next one working on this lol

ridge -> build poly -> predict label: Ca marche mais pas très bons résultats (mieux si tu montes le degré mais tres bas F1)

least squares -> predict label : marche pas avec build poly pck pas les bonnes dims (build poly rajoute une colonne, donc impossible de faire data@weight) Ca marche sans, mais on a que 1 valeur a 1, le reste a -1 (et ya pas de params a changer) F1 SCORE 0.000 ACCURACY 0.912

mean squared gd -> (same thing pour build poly) les valeurs sont fucked up mdr [1.13497280e+79 1.72690226e+79 3.20689183e+78 ... 1.36781180e+78 1.99141883e+78 1.76606508e+78] c'est du 10^78 a peu pres, donc ya tout qui fini par etre a 1 et rien a -1 

mean squared sgd -> same as gd mais on est plutot dans du 10^14, et que des negatifs

J'ai aussi tenté ridge en filtrant les données d'une autre manière, aka juste prendre les colonnes qui ont plus que 70% de données (par rapport aux nan) et ca me donne des meilleurs résultats... snif snif donc F1 SCORE 0.139 ACCURACY 0.837 Après j'ai rien clean up or anything donc c'est un peu ridicule genre ya toutes les valeurs d'exceptions encore (en vrai j'ai aucune idée de pourquoi ca a marché ??? Pck j'ai pas enlevé les nan ?? Donc ca a du bader hahahaha)

Si je mets 1000 max_iters ca overflow error, donc j'ai laissé 100, a tester avec 500 par exemple

Idées: Peut être que normalizer les data ca change qqch mour les deux mean squares ? 

Questions TA:
- regarder nos logisitc methods
- pk ca marche pas lol
- est - ce que c'est une bonne idée la manière dont on fait ou est-ce qu'il nous faut beaucoup plus de colonnes



### noms des variables et valeurs utilisées

- cleaned_x_train/test : data nettoyée avec nos 28 colonnes choisies
- x_train/test_filtered : data avec enlevé ceux avec moins de 70% de donnees

- degres: pour ridge utiliser un grand chiffre, pour le reste utiliser 1 (bah en fait c'est juste pour build poly donc pas utiliser (je crois, j'ai pas trouvé de valuers qui marche))

- max_iters: j'ai l'impression que plus je mets grand, plus mes y_pred sont grands (why??????) (avec 1, j'ai que des -1.3 environ, quand je mets 2 j'ai entre 1 et 20, quand je mets 100 j'ai 10^78 lol) Ca explique le overflow error quand je mets 1000 i guess






In [177]:
yp = predict_labels(w,cleaned_x_test)

[19.03807512 18.15791472  4.53047456 ...  2.28483186  2.90662102
  2.94509189]


In [175]:
print(len(np.where(yp == 1)[0]))

0


#### CSV Submission

In [141]:
create_csv_submission(test_ids,yp,"outputs/ridge_degre25_200features.csv")